# Exercício 07_02

#### Juliana Carolina Thuler dos Santos

## KNN

Alterar a função `vizinho_mais_proximo_()` para identificar se a variável resposta é contínua ou não, fazendo um KNN Classificador ou KNN Regressor de acordo com isso.

Para o KNN Classificador, retornar a probabilidade de pertencer a uma determinada classe. Já para o KNN Regressor, trazer uma tupla com a média ou mediana, de acordo com o parâmetro `est`, e o desvio padrão.

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import mode

In [2]:
df = pd.read_csv('iris.csv')

In [3]:
from sklearn.model_selection import train_test_split as tts

df_treino, df_teste = tts(df, test_size = 0.2, random_state = 42)

In [4]:
sample_teste_continuo_1 = pd.DataFrame(df_teste.iloc[0,:]).T.drop(columns = [df.columns[0]])
sample_teste_continuo_2 = pd.DataFrame(df_teste.iloc[0,:]).T.drop(columns = [df.columns[1]])
sample_teste_continuo_3 = pd.DataFrame(df_teste.iloc[0,:]).T.drop(columns = [df.columns[2]])
sample_teste_continuo_4 = pd.DataFrame(df_teste.iloc[0,:]).T.drop(columns = [df.columns[3]])
sample_teste_classif = pd.DataFrame(df_teste.iloc[0,:]).T.drop(columns = ['species'])

In [5]:
def vizinho_mais_proximo_(d_treino, nova_instancia, var_resposta, k = 1, est = 'média'):
    
    '''
    Dado um conjunto de dados chamado d_treino, calcula a predição da classe para a nova_instancia.
    '''
    
    list_col = []
    list_col_uniques = []
    
    d_treino_aux = d_treino.copy()
    
    for col in d_treino_aux.columns:
        
        if d_treino_aux[col].dtype == 'O':
            
            list_aux = list(d_treino_aux[col].unique())

            list_col.append(col)
            list_col_uniques.append(list_aux)
                        
            d_treino_aux[col] = d_treino_aux[col].replace(list_aux, [i for i in range(len(list_aux))])
            try:
                nova_instancia[col] = nova_instancia[col].replace(list_aux, [i for i in range(len(list_aux))])
            except:
                pass
    
    lista_distancia = []
    
    for lin in range(d_treino_aux.shape[0]):
        
        d_calc = d_treino_aux.drop(columns = [var_resposta]) #excluindo variável resposta para calcular distância
        
        d = 0
        
        d = np.sqrt(np.sum((d_calc.iloc[lin,:].values - nova_instancia.values)**2))
        
        lista_distancia.append((d, d_treino_aux.iloc[lin][var_resposta]))
    
    lista_distancia = sorted(lista_distancia)
    lista_distancia = lista_distancia[:k]
                  
    if d_treino[var_resposta].dtype == 'float':
                  
        vetor = np.array([i[1] for i in lista_distancia])
                  
        if est == 'média':
                  
            media = round(np.mean(vetor),2)
            desv_pad = round(np.std(vetor),2)
            
            resposta = 'média: ' + str(media) + ' desvio padrão: ' + str(desv_pad)
        
        if est == 'mediana':
                  
            mediana = round(np.median(vetor),2)
            desv_pad = round(np.std(vetor),2)
            
            resposta = 'mediana: ' + str(mediana) + ' desvio padrão: ' + str(desv_pad)
            
        if est == 'moda':
                  
            moda = mode(vetor).mode[0]
            desv_pad = round(np.std(vetor),2)
            
            resposta = 'moda: ' + str(moda)+ ' desvio padrão: ' + str(desv_pad)
                  
    else:

        moda = int(mode([i[1] for i in lista_distancia]).mode[0])
        
        index = list_col.index(var_resposta)
        lista_transform = list_col_uniques[index]
        
        resposta = 'Classificação: ' + lista_transform[moda]
            
    return resposta

In [6]:
#linha usada para testar função
pd.DataFrame(df_teste.iloc[0,:]).T

,sepal_length,sepal_width,petal_length,petal_width,species
73,6.1,2.8,4.7,1.2,versicolor


In [7]:
vizinho_mais_proximo_(df_treino, sample_teste_classif, 'species', k = 5)

'Classificação: versicolor'

In [8]:
vizinho_mais_proximo_(df_treino, sample_teste_continuo_1, df_treino.columns[0], k = 5)

'média: 6.16 desvio padrão: 0.39'

In [9]:
vizinho_mais_proximo_(df_treino, sample_teste_continuo_2, df_treino.columns[1], k = 5,est = 'média')

'média: 2.72 desvio padrão: 0.27'

In [10]:
vizinho_mais_proximo_(df_treino, sample_teste_continuo_3, df_treino.columns[2], k = 5,est = 'mediana')

'mediana: 4.3 desvio padrão: 0.25'

In [11]:
vizinho_mais_proximo_(df_treino, sample_teste_continuo_4, df_treino.columns[3], k = 5, est = 'moda')

'moda: 1.4 desvio padrão: 0.07'